In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import zipfile
import wget

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Scrape the Mars News Site
url = "https://s3.amazonaws.com/tripdata/index.html"
browser.visit(url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
data=[]
for a in soup.find_all('a', href=True):
    data.append(a['href'])
data

In [ ]:
#breakout into Year Buckets
y2017_y2022 = []
y2017 = []
y2018 = []
y2019 = []
y2020 = []
y2021 = []
y2022 = []


for i in range(0, len(data) -1):
    date = data[i].split('/')[-1].split('-')[0]
    
    if (date != "JC"):
        year = int(date[0:4])
        if (year == 2017):
            y2017.append(data[i])
        elif (year == 2018):
            y2018.append(data[i])
        elif (year == 2019):
            y2019.append(data[i])
        elif (year == 2020):
            y2020.append(data[i])
        elif (year == 2021):
            y2021.append(data[i])
        elif (year == 2022):
            y2022.append(data[i])
 #Pull all the Items in a certain date range           
for i in range(0, len(data) -1):
    date = data[i].split('/')[-1].split('-')[0]
    if (date != "JC"):
        year = int(date[0:4])
        if (year >= 2021):
            y2017_y2022.append(data[i])


        


In [ ]:
# unzip the files of the date range
dfile = y2017_y2022  
ziplist=[]
for i in dfile:
    ziplist.append(wget.download(i,"zip"))
    

In [ ]:
# convert the File name to .csv
print(ziplist)
csvlist = []
for i in range(0, len(ziplist)):
    csvlist.append(ziplist[i].split('/')[-1].rstrip(".zip"))
    
csvlist

In [ ]:

# with zipfile.ZipFile("zip/202203-citibike-tripdata.csv.zip") as z:
#     with z.open("202203-citibike-tripdata.csv") as f:
#                citi1 = pd.read_csv(f, header=0, delimiter="\t")
# print(citi1.head()) # print the first 5 rows
# # os.path.split

In [ ]:

# unzip files
zf=[]

for x in range(0,len(ziplist)):
    zf.append(zipfile.ZipFile(ziplist[x]))
    # df = pd.read_csv(zf.open(ziplist[x].split('/')[-1].rstrip(".zip")))
    # zf.append(zipfile.Zipfile(ziplist[x]))
# if you want to see all files inside zip folder
zf


In [ ]:
# df = pd.read_csv(zf.open(csvlist[x]))
# df.to_csv(f"csv/{csvlist[x]}") 

In [ ]:

# now read your csv file # remeber that
for x in range(0, len(csvlist)):
    df = pd.read_csv(zf[x].open(csvlist[x]))
    df = df.dropna()
    
    df.to_csv(f"csv/{csvlist[x]}") 

In [ ]:
df.head()

In [ ]:
# 0
# zf = zipfile.ZipFile("zip/202203-citibike-tripdata.csv.zip") 
# # if you want to see all files inside zip folder
# zf

# now read your csv file 
df = pd.read_csv(zf.open("202203-citibike-tripdata.csv"))
df.head()
df.to_csv('csv/202203-citibike-tripdata.csv') 

In [2]:
#Merge the three files
maindf = pd.concat(
    map(pd.read_csv, ['csv/202106-citibike-tripdata.csv','csv/202107-citibike-tripdata.csv','csv/202108-citibike-tripdata.csv']), ignore_index=True)
maindf = maindf.dropna()
maindf.head()


/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/concat.py:294: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(


,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0,4B923B054861DAEF,docked_bike,2021-06-01 23:12:34,2021-06-01 23:14:46,Driggs Ave & N 9 St,5411.08,Bayard St & Leonard St,5442.05,40.718169,-73.955201,40.719156,-73.948854,member
1,1,7F5D78629F671C01,docked_bike,2021-06-16 17:14:56,2021-06-16 17:29:15,Fulton St & Broadway,5175.08,Mercer St & Spring St,5532.01,40.711066,-74.009447,40.723627,-73.999496,casual
2,2,3F11FC94B816A370,docked_bike,2021-06-07 19:41:55,2021-06-07 19:51:28,Devoe St & Lorimer St,5259.06,Manhattan Av & Leonard St,5442.09,40.713352,-73.949103,40.720840,-73.948440,casual
3,3,9116683DA18320D9,electric_bike,2021-06-17 15:13:15,2021-06-17 15:33:25,Driggs Ave & N 9 St,5411.08,Greenwich Ave & Charles St,5914.08,40.718169,-73.955201,40.735238,-74.000271,member
4,4,FBBF59A0A03AC3AF,electric_bike,2021-06-18 08:27:03,2021-06-18 08:53:37,Graham Ave & Conselyea St,5291.05,E 30 St & Park Ave S,6206.08,40.715143,-73.944507,40.744449,-73.983035,member


In [3]:
startdatedf=pd.DataFrame(maindf['start_station_name'])
# 
startdatedf.head()
len(startdatedf)

9296859

In [4]:
enddatedf=pd.DataFrame(maindf['end_station_name'])
enddatedf.rename(columns = {'end_station_name':'start_station_name'}, inplace = True)
enddatedf.head()
len(enddatedf)

9296859

In [5]:
frames=[startdatedf,enddatedf]
stationname=pd.concat(frames)
len(stationname)


18593718

In [19]:
stationname=stationname['start_station_name'].drop_duplicates()


KeyError: 'start_station_name'

In [20]:
stationname

0                Driggs Ave & N 9 St
1               Fulton St & Broadway
2              Devoe St & Lorimer St
4          Graham Ave & Conselyea St
5                Clark St & Henry St
                     ...            
8161135                 Newport PATH
8177180                   Bergen Ave
8255794             Essex Light Rail
8762279                Grove St PATH
9076804              Jersey & 6th St
Name: start_station_name, Length: 1558, dtype: object

In [21]:
stationname=pd.DataFrame(stationname)
stationname

,start_station_name
0,Driggs Ave & N 9 St
1,Fulton St & Broadway
2,Devoe St & Lorimer St
4,Graham Ave & Conselyea St
5,Clark St & Henry St
...,...
8161135,Newport PATH
8177180,Bergen Ave
8255794,Essex Light Rail
8762279,Grove St PATH


In [22]:
stationname.to_csv('csv/stationname.csv') 